# Creacion de transformacdores y PPipelines Personalizados. 

En este notebook se  muestra la creacion de tranformadores y Pipelines personalizados.

#### DataSet
## Descripcion
ISCX NSL-KDD dataset 2009 

We apologize, this dataset is no longer available.

ISCX NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in [1]. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods.

Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

## Data files

* <span style="color:green">KDDTrain+.ARFF: The full NSL-KDD train set with binary labels in ARFF format</span>
* KDDTrain+.TXT: The full NSL-KDD train set including attack-type labels and difficulty level in CSV format
* KDDTrain+_20Percent.ARFF: A 20% subset of the KDDTrain+.arff file
* KDDTrain+_20Percent.TXT: A 20% subset of the KDDTrain+.txt file
* KDDTest+.ARFF: The full NSL-KDD test set with binary labels in ARFF format
* KDDTest+.TXT: The full NSL-KDD test set including attack-type labels and difficulty level in CSV format
* KDDTest-21.ARFF: A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21
* KDDTest-21.TXT: A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21
Improvements to the KDD'99 dataset

The ISCX NSL-KDD data set has the following advantages over the original KDD data set:

It does not include redundant records in the train set, so the classifiers will not be biased towards more frequent records.
There is no duplicate records in the proposed test sets; therefore, the performance of the learners are not biased by the methods which have better detection rates on the frequent records.
The number of selected records from each difficultylevel group is inversely proportional to the percentage of records in the original KDD data set. As a result, the classification rates of distinct machine learning methods vary in a wider range, which makes it more efficient to have an accurate evaluation of different learning techniques.
The number of records in the train and test sets are reasonable, which makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research works will be consistent and comparable.

Statistical observations

One of the most important deficiencies in the KDD data set is the huge number of redundant records, which causes the learning algorithms to be biased towards the frequent records, and thus prevent them from learning unfrequent records which are usually more harmful to networks such as U2R and R2L attacks. In addition, the existence of these repeated records in the test set will cause the evaluation results to be biased by the methods which have better detection rates on the frequent records.

In addition, we analyzed the difficulty level of the records in KDD data set. Surprisingly, about 98% of the records in the train set and 86% of the records in the test set were correctly classified with all the 21 learners.

In order to perform our experiments, we randomly created three smaller subsets of the KDD train set each of which included fifty thousand records of information. Each of the learners where trained over the created train sets. We then employed the 21 learned machines (7 learners, each trained 3 times) to label the records of the entire KDD train and test sets, which provides us with 21 predicated labels for each record. Further, we annotated each record of the data set with a #successfulPrediction value, which was initialized to zero. Now, since the KDD data set provides the correct label for each record, we compared the predicated label of each record given by a specific learner with the actual label, where we incremented #successfulPrediction by one if a match was found. Through this process, we calculated the number of learners that were able to correctly label that given record. The highest value for #successfulPrediction is 21, which conveys the fact that all learners were able to correctly predict the label of that record.

Statistics of redundant records in the KDD train set

Original records | Distinct records | Reduction rate

Attacks: 3,925,650 | 262,178 | 93.32%
Normal: 972,781 | 812,814 | 16.44%
Total: 4,898,431 | 1,074,992 | 78.05%
Statistics of redundant records in the KDD test set

Original records | Distinct records | Reduction rate

Attacks: 250,436 | 29,378 | 88.26%
Normal: 60,591 | 47,911 | 20.92%
Total: 311,027 | 77,289 | 75.15%
License

You may redistribute, republish, and mirror the ISCX NSL-KDD dataset in any form. However, any use or redistribution of the data must include a citation to the NSL-KDD dataset and the paper referenced below.

References: [1] M. Tavallaee, E. Bagheri, W. Lu, and A. Ghorbani, “A Detailed Analysis of the KDD CUP 99 Data Set,” Submitted to Second IEEE Symposium on Computational Intelligence for Security and Defense Applications (CISDA), 2009.
[URL] (https://www-unb-ca.translate.goog/cic/datasets/nsl.html?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc)



## Imports

In [32]:
import arff     
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler


## Funciones Auxiliares



In [33]:
def load_kdd_dataset(data_path):
    """Lectura del Dataset NSL-KDD"""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
        attributes = [attr[0] for attr in dataset['attributes']]
        return pd.DataFrame(dataset["data"], columns=attributes)

In [34]:
# Construccion de uuna funcion que realice el particionado completo. 
def train_val_test_split(df, rstate= 42, shuffle=True, stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_Set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_Set[stratify] if stratify else None 
    val_set, test_Set = train_test_split(
        test_Set, test_size=0.5,  random_state=rstate, shuffle=shuffle, stratify=strat)
    return(train_set, val_set, test_Set)

## LECTURA DEL DATASET

In [35]:
df = load_kdd_dataset(r"C:\Users\yanet\OneDrive\Documentos\ANTLR\datasets\NSL-KDD/KDDTrain+.arff")

## DIVISION DEL DATASET

In [36]:
train_val_test_split

train_set, val_set, test_set = train_val_test_split(df, stratify='protocol_type')

In [37]:
print("longitud del Training Set:", len(train_set))
print("longitud del Validation Set:", len(val_set))
print("longitud del Test Set:", len(test_set))

longitud del Training Set: 75583
longitud del Validation Set: 25195
longitud del Test Set: 25195


## API'S de Sklearn 

Antes de continuar es necesario hacer una pequeña reseña sobre como funcionan las API'S de Sklearn:

* ***Estimators**: Cualquier objeto que pueda estimar algun parametro:

    * El  propio estimador se forman medinate el metodo fit(), que siempre toma un dataset como argumento.
    * Cualquier otro parametro de este metodo, es un hiperparametro.

* ***Transformers**: Son estimadores capaces de transformar el DataSet  (como imputer).
    * La transformacion se realiza mediante el metodo transform().
    * Reciben un DataSet como parametro de entrada.

* ***Predictrors**: Son estimadores capaces de realizar predicciones:

    * La prediccion se realiza mediante ell metodo predict().
    * Reciben un DataSet como entrada.
    * Retornan  un DataSet como predicicone.
    * Tienen un metodo score( )para evaluar resultado de la prediccion. 

## 1.- Construyendo tranformadores personalizados. 

La creacion de transformadores propios permite mantener el codigo mucho mas limpio yb estructurado a la hora de preparar los datos para los algoritmos de ML. Ademas, facilita la utilizacion de codigo para otros proyectos. 

Antes de comenzar, vamos a recperar el DataSet limpio y vamos a separar las etiquetas del resto de los datos, no necesariamente se requiere aplcar las mismas las tranformaciones en ambos conjuntos. 

In [38]:
X_train = train_set.drop("class", axis=1)
y_train = train_set["class"].copy

In [39]:
## PARA FACILITAR ESTA SECCION ES NECESARIO AÑADIR ALGUNOS VALORES NLOS A ALGUNAS CARACTERISTICAS DEL DATASET.
X_train.loc[(X_train["src_bytes"]>400) &(X_train["src_bytes"] < 800), "src_bytes"] = np.nan
X_train.loc[(X_train["src_bytes"]>500) &(X_train["dst_bytes"] < 2000), "dst_bytes"] = np.nan
X_train


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,tcp,http,SF,NaN,53508.0,0,0.0,0.0,0.0,...,9.0,255.0,1.00,0.00,0.11,0.03,0.00,0.00,0.0,0.0
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64559,0.0,tcp,systat,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.0
67272,0.0,tcp,http,SF,210.0,736.0,0,0.0,0.0,0.0,...,119.0,255.0,1.00,0.00,0.01,0.02,0.02,0.01,0.0,0.0
32452,3.0,tcp,smtp,SF,889.0,NaN,0,0.0,0.0,0.0,...,111.0,155.0,0.64,0.04,0.01,0.01,0.01,0.00,0.0,0.0
112657,0.0,tcp,http,SF,284.0,444.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


## T2.- Transformadores para atributos numericos

In [40]:
#Transfomador creado para eliminar las filas con valores nulos

from sklearn.base import BaseEstimator, TransformerMixin

class DeleteNanRows(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        return X.dropna()
    

In [41]:
delete_nan = DeleteNanRows()
X_train_prep = delete_nan.fit_transform(X_train)
X_train_prep

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
98007,0.0,udp,domain_u,SF,46.0,139.0,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90665,0.0,tcp,ftp_data,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,63.0,0.25,0.02,0.02,0.00,1.00,1.00,0.0,0.0
64559,0.0,tcp,systat,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.0
67272,0.0,tcp,http,SF,210.0,736.0,0,0.0,0.0,0.0,...,119.0,255.0,1.00,0.00,0.01,0.02,0.02,0.01,0.0,0.0
112657,0.0,tcp,http,SF,284.0,444.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [42]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler
import pandas as pd

class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.scaler = RobustScaler()
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns])
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy[self.columns] = self.scaler.transform(X[self.columns])
        return X_copy

In [43]:
custom_scaler = CustomScaler(["src_bytes", "dst_bytes"])
X_train_prep = custom_scaler.fit_transform(X_train_prep)
X_train_prep.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.0,1.0,0.0,0.0
108116,0.0,tcp,http,SF,1.328829,0.832461,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.0,0.0,0.0,0.0
89913,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.0,1.0,0.0,0.0
106319,0.0,icmp,eco_i,SF,-0.004505,0.000000,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.0,0.0,0.0,0.0
98007,0.0,udp,domain_u,SF,0.166667,0.181937,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
100052,0.0,tcp,http,SF,0.887387,1.952880,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
28800,0.0,tcp,ftp_data,SF,1.463964,0.000000,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.0,0.0,0.0,0.0
69315,0.0,tcp,systat,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
100360,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,13.0,0.05,0.07,0.00,0.00,1.0,1.0,0.0,0.0
29208,0.0,tcp,http,SF,1.472973,5.606021,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0


In [44]:
X_train_prep.head(20)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.0,1.0,0.0,0.0
108116,0.0,tcp,http,SF,1.328829,0.832461,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.0,0.0,0.0,0.0
89913,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.0,1.0,0.0,0.0
106319,0.0,icmp,eco_i,SF,-0.004505,0.000000,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.0,0.0,0.0,0.0
98007,0.0,udp,domain_u,SF,0.166667,0.181937,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
100052,0.0,tcp,http,SF,0.887387,1.952880,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
28800,0.0,tcp,ftp_data,SF,1.463964,0.000000,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.0,0.0,0.0,0.0
69315,0.0,tcp,systat,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
100360,0.0,tcp,private,S0,-0.040541,0.000000,0,0.0,0.0,0.0,...,255.0,13.0,0.05,0.07,0.00,0.00,1.0,1.0,0.0,0.0
29208,0.0,tcp,http,SF,1.472973,5.606021,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0


In [45]:
#Transformador para codificar unicamente las colunas categoricas y 
#Devolver un DataFrame
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

class CustomOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, sparse=False):
        self._oh = OneHotEncoder(sparse=sparse, handle_unknown='ignore')
        self._feature_names = None

    def fit(self, X, y=None):
        x_cat = X.select_dtypes(include=['object'])
        self._oh.fit(x_cat)
        self._feature_names = self._oh.get_feature_names_out(x_cat.columns)
        return self

    def transform(self, X, y=None):
        x_copy = X.copy()
        x_cat = x_copy.select_dtypes(include=['object'])
        x_num = x_copy.select_dtypes(exclude=['object'])
        # Transformar categóricas y crear DataFrame
        x_cat_oh = self._oh.transform(x_cat)
        x_cat_oh_df = pd.DataFrame(x_cat_oh, columns=self._feature_names, index=x_copy.index)
        # Concatenar numéricos y categóricos
        return pd.concat([x_num, x_cat_oh_df], axis=1)

## 6.- Construyendo Pipelines personalizados

Los Pipelines nos permiten agrupar en un flujo de ejecución todas las operaciones de transformación que se necesitan realizar sobre un conjunto de datos, esto facilita mucho las transformaciones para diferentes conjuntos de datos. Los puntos a tener en cuenta son:

* recibe un conjunto de datos pares (Nombre, estimador)
* Todos menos el último deben ser transformadores.
* El pipeline expone los mismos métodos que **el último transformador**.
* Cunado se llama al método fit() del pipeline, se llama secuencialmente al método fit_transform() de los estimadores y se les pasa de manera secuencial el ouput del anterior como input del siguiente. El último invoca el método fit().

In [46]:
# Construcción de un pipeline para los atributos númericos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('rbst_scaler', RobustScaler()),
])

In [47]:
#La clase inputer no admite valores categoricos , se elminan los atributos categoricos
#La clase imputer no admite valores categóricos, se eliminan los atributos categóricos
X_train_num = X_train.select_dtypes(exclude=['object'])

X_train_prep = num_pipeline.fit_transform(X_train_num)
X_train_prep = pd.DataFrame(X_train_prep, columns=X_train_num.columns, index=X_train_num.index)
X_train_prep.head(10)


,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,0.000000,106.377734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.421965,0.787755,0.515789,-0.428571,1.833333,1.5,0.00,0.00,0.0,0.0
31899,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.236735,-0.515789,0.285714,0.000000,0.0,1.00,1.00,0.0,0.0
108116,0.0,1.056680,1.264414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.248555,0.787755,0.515789,-0.428571,0.500000,3.0,0.00,0.00,0.0,0.0
89913,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.191837,-0.473684,0.571429,0.000000,0.0,1.00,1.00,0.0,0.0
106319,0.0,-0.141700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.462428,-0.224490,0.515789,-0.428571,16.666667,28.5,0.00,0.00,0.0,0.0
98007,0.0,0.012146,0.276342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.783673,0.515789,-0.285714,0.000000,0.0,0.00,0.00,0.0,0.0
16447,0.0,7.072874,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.658960,0.306122,0.042105,0.142857,0.166667,0.5,0.00,0.00,0.0,0.0
64957,1.0,0.000000,0.654076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.329480,0.485714,0.147368,0.000000,0.166667,0.5,0.02,0.02,0.0,0.0
100052,0.0,0.659919,2.966203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.515789,-0.428571,0.000000,0.0,0.00,0.00,0.0,0.0
28800,0.0,1.178138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.427746,-0.138776,0.515789,-0.428571,16.666667,5.5,0.00,0.00,0.0,0.0


In [48]:
X_train_num.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,NaN,53508.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,255.0,1.00,0.00,0.11,0.03,0.00,0.00,0.0,0.0
31899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,304.0,636.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
98007,0.0,46.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0
16447,0.0,1790.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,141.0,137.0,0.55,0.04,0.01,0.01,0.00,0.00,0.0,0.0
64957,1.0,NaN,329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,198.0,181.0,0.65,0.03,0.01,0.01,0.02,0.02,0.0,0.0
100052,0.0,206.0,1492.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
28800,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.00,0.00,0.0,0.0


In [49]:
X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,0.000000,106.377734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.421965,0.787755,0.515789,-0.428571,1.833333,1.5,0.00,0.00,0.0,0.0
31899,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.236735,-0.515789,0.285714,0.000000,0.0,1.00,1.00,0.0,0.0
108116,0.0,1.056680,1.264414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.248555,0.787755,0.515789,-0.428571,0.500000,3.0,0.00,0.00,0.0,0.0
89913,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.191837,-0.473684,0.571429,0.000000,0.0,1.00,1.00,0.0,0.0
106319,0.0,-0.141700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.462428,-0.224490,0.515789,-0.428571,16.666667,28.5,0.00,0.00,0.0,0.0
98007,0.0,0.012146,0.276342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.783673,0.515789,-0.285714,0.000000,0.0,0.00,0.00,0.0,0.0
16447,0.0,7.072874,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.658960,0.306122,0.042105,0.142857,0.166667,0.5,0.00,0.00,0.0,0.0
64957,1.0,0.000000,0.654076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.329480,0.485714,0.147368,0.000000,0.166667,0.5,0.02,0.02,0.0,0.0
100052,0.0,0.659919,2.966203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.515789,-0.428571,0.000000,0.0,0.00,0.00,0.0,0.0
28800,0.0,1.178138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.427746,-0.138776,0.515789,-0.428571,16.666667,5.5,0.00,0.00,0.0,0.0


A continuacion se presenta el metodo ColumnTransformer, que ejecuta todos los Pipelines en paralelo y concatea el resultados  , para ellos los resultados de los pipelines debe de ser numerico 

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

num_attributes = list(X_train.select_dtypes(exclude=['object']).columns)
cat_attributes = list(X_train.select_dtypes(include=['object']).columns)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attributes),
    ("cat", OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_attributes),
])


In [51]:
# Aplicar el pipeline
x_train_prep = full_pipeline.fit_transform(X_train)

# Obtener nombres de columnas correctamente
cat_feature_names = full_pipeline.named_transformers_['cat'].get_feature_names_out(cat_attributes)
all_feature_names = list(num_attributes) + list(cat_feature_names)

x_train_prep = pd.DataFrame(x_train_prep, columns=all_feature_names)

In [52]:
X_train.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,tcp,http,SF,NaN,53508.0,0,0.0,0.0,0.0,...,9.0,255.0,1.00,0.00,0.11,0.03,0.00,0.00,0.0,0.0
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
98007,0.0,udp,domain_u,SF,46.0,139.0,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0
16447,0.0,tcp,smtp,SF,1790.0,NaN,0,0.0,0.0,0.0,...,141.0,137.0,0.55,0.04,0.01,0.01,0.00,0.00,0.0,0.0
64957,1.0,tcp,smtp,SF,NaN,329.0,0,0.0,0.0,0.0,...,198.0,181.0,0.65,0.03,0.01,0.01,0.02,0.02,0.0,0.0
100052,0.0,tcp,http,SF,206.0,1492.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
28800,0.0,tcp,ftp_data,SF,334.0,0.0,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.00,0.00,0.0,0.0
